# Dual Augmented Lagrangian
1. [Github](https://github.com/yaoxuexa/CNNCS/tree/master/dal-master)

In [ ]:
# % dal - dual augmented Lagrangian method for sparse learaning/reconstruction
# %
# % Overview:
# %  Solves the following optimization problem
# %   xx = argmin f(x) + lambda*c(x)
# %  where f is a user specified (convex, smooth) loss function and c
# %  is a measure of sparsity (currently L1 or grouped L1)
# %
# % Syntax:
# %  [ww, uu, status] = dal(prob, ww0, uu0, A, B, lambda, <opt>)
# %
# % Inputs:
# %  prob   : structure that contains the following fields:
# %   .obj      : DAL objective function
# %   .floss    : structure with three fields (p: primal loss, d: dual loss, args: arguments to the loss functions)
# %   .fspec    : function handle to the regularizer spectrum function 
# %               (absolute values for L1, vector of norms for grouped L1, etc.)
# %   .dnorm    : function handle to the conjugate of the regularizer function
# %               (max(abs(x)) for L1, max(norms) for grouped L1, etc.)
# %   .softth   : soft threshold function
# %   .mm       : number of samples (scalar)
# %   .nn       : number of unknown variables (scalar)
# %   .ll       : lower constraint for the Lagrangian multipliers ([mm,1])
# %   .uu       : upper constraint for the Lagrangian multipliers ([mm,1])
# %   .Ac       : inequality constraint Ac*aa<=bc for the LMs ([pp,mm])
# %   .bc       :                                             ([pp,1])
# %   .info     : auxiliary variables for the objective function
# %   .stopcond : function handle for the stopping condition
# %   .hessMult : function handle to the Hessian product function (H*x)
# %   .softth : function handle to the "soft threshold" function
# %  ww0    : initial solution ([nn,1)
# %  uu0    : initial unregularized component ([nu,1])
# %  A          : struct with fields times, Ttimes, & slice.
# %   .times    : function handle to the function A*x.
# %   .Ttimes   : function handle to the function A'*y.
# %   .slice    : function handle to the function A(:,I).
# %  B      : design matrix for the unregularized component ([mm,nu])
# %  lambda : regularization constant (scalar)
# %  <opt>  : list of 'fieldname1', value1, 'filedname2', value2, ...
# %   aa        : initial Lagrangian multiplier [mm,1] (default zero(mm,1))
# %   tol       : tolerance (default 1e-3)
# %   maxiter   : maximum number of outer iterations (default 100)
# %   eta       : initial barrier parameter (default 1)
# %   eps       : initial internal tolerance parameter (default 1e-4)
# %   eta_multp : multiplying factor for eta (default 2)
# %   eps_multp : multiplying factor for eps (default 0.5)
# %   solver    : internal solver. Can be either:
# %               'nt'   : Newton method with cholesky factorization
# %               'ntsv' : Newton method memory saving (slightly slower)
# %               'cg'   : Newton method with PCG (default)
# %               'qn'   : Quasi-Newton method
# %   display   : display level (0: none, 1: only the last, 2: every
# %               outer iteration, (default) 3: every inner iteration)
# %   iter      : output the value of ww at each iteration 
# %               (boolean, default 0)
# % Outputs:
# %  ww     : the final solution
# %  uu     : the final unregularized component
# %  status : various status values
# %
# % Reference:
# % "Super-Linear Convergence of Dual Augmented Lagrangian Algorithm
# % for Sparse Learning."
# % Ryota Tomioka, Taiji Suzuki, and Masashi Sugiyama. JMLR, 2011. 
# % "Dual Augmented Lagrangian Method for Efficient Sparse Reconstruction"
# % Ryota Tomioka and Masashi Sugiyama
# % http://arxiv.org/abs/0904.0584
# % 
# % Copyright(c) 2009-2011 Ryota Tomioka
# % This software is distributed under the MIT license. See license.txt

In [ ]:
import time
import numpy as np

In [ ]:
def dalsql1(ww, A, bb, lambdaa):
    # opt = propertylist2struct(varargin{:});
    # opt = set_defaults(opt,'solver','cg','stopcond','pdg');
    prob.floss = struct('p', @loss_sqp, 'd', @loss_sqd, 'args', {{bb}})
    prob.fspec = lambda xx : abs(xx)
    prob.dnorm = lambda vv : max(abs(vv))
    prob.obj = @objdall1
    probj.softh = @l1_softh
    prob.stopcond = opt.stopcond
    prob.ll = -np.inf * np.ones(bb.shape)
    prob.uu = np.inf * np.ones(bb.shape)
    prob.Ac = []
    prob.bc = []
    prob.info = []
    
    if opt.solver == 'cg':
        prob.hessMult = @hessMultdall1
    
    if opt.stopconf = 'fval':
        opt.feval = 1
    
    if isnumeric(A):
        A = A[:,:]
        mm, nn = A.shape
        fA = struct('times', lambda x: A*x, 'Ttimes', lambda x: A.T*x, 'slice', lambda I = A[:,I])
    elif iscell(A):
        mm = A{3}
        nn = A{4}
        fAslice = lambda I : fA(sparse)...
    else:
        print ('A must either be numeric or cell')
    
    prob.mm = mm
    pro.nn = nn
    
    opt.display = 0
    ww, uu, status = dal(prob, ww, [], fA, [], lambdaa, opt)

def dal(prob, ww0, uu0, A, B, lambdaa):
    opt = {'tol' : 0.001, 'iterr':0, 'maxiter':1000, 'eta':[], 'eps':1, 'eps_multp':-.99, 'eta_multp':2
                   'solver': 'cg', 'boostb':1, 'display':2 } 
    
    prob = { 'll' : -np.inf * np.ones((prob.mm, 1)), 
             'uu' : np.inf * np.ones((prob.mm, 1)),
             'Ac' : [],
             'bc' : [],
             'info' : [],
             'finddir' : []
           }
    if len(opt.eta) == 0:
        opt.eta = 0.01/ lambdaa
    
    if len(uu0) == 0 and len(opt.eta) == 1:
        opt.eta = opt.eta * [1, 1]
    
    if len(uu0) == 0 and len(opt.eta_multp) < 2:
        opt.eta_multp = opt.eta_multp * [1,1]
    
    if opt.display > 0:
        if len(uu0) == 0:
            nuu = len(uu0)
            vstr = str(prob.nn) + ' + ' + str(nuu)
        else:
            vstr = str(prob.nn)
        
        lstr = str(prob.floss.p)
        lstr = lstr[5:-1]
        print ('DAL ver1.05\n')
        print ('#samples = ', prob.mm, ' #variables = ', vstr, ' #lambda = ', lambdaa, ' #loss = ', lstr, ' #solver = ', opt.solver)
    
    if opt.iterr:
        nwu  = len(ww0[:]) + len(uu0[:])
        xx = np.hstack(( np.vstack((ww0[:], uu0[:])), np.ones((nwu, opt.maxiter - 1) * np.nan) ))
    
    res = np.nan * np.ones((1, opt.maxiter))
    fval = np.nan * np.ones((1, opt.maxiter))
    etaout = np.nan * np.ones((len(opt.eta), opt.maxiter))
    time = np.nan * np.ones((1, opt.maxiter))
    xi = np.nan * np.ones((1, opt.maxiter))
    num_pcg = np.nan * np.ones((1, opt.maxiter))
    
    time0 = time.time()
    ww = ww0
    uu = uu0
    gtmp = np.zeros(ww.shape)
    
    if len(opt.aa):
        ff, gg = evalloss(prob, ww, uu, A, B)
        aa = -gg
        if np.any(aa = prob.ll) || np.any(aa == prob.uu):
            print ('Invalid initial solution; using instead using ww = np.zeros((n,1))/n')
            w0 = np.zeros((prob.nn, 1))
            ff, gg = evalloss(prob, w0, uu, A, B)
            aa = -gg
    
    else:
        aa = opt.aa
    
    dval = np.inf
    eta = opt.eta
    epsl = opt.eps
    info = prob.info
    info.solver = opt.solver
    info.ATaa = []
    spec = prob.fspec(ww)
    
    for ii in range(maxiter):
        ww_old = ww
        uu_old = uu
        etaout[:,ii] = eta.T
        time[ii] = time.time() - time0
        
        # Evaluate objectionve and check stopping condition
        fval[ii] = evalprim(prob, ww, uu, A, B, lambdaa)
        
        if prob.stopcond == 'pdg':
            dval = min(dval, evaldual(prob, aa, A, B, lambdaa))
            res[ii] = (fval[ii] - (-dval)) / fval[ii]
            ret = res[ii] < opt.tol
        elif prob.stopcond == 'fval':
            res[ii] = fval[ii] - opt.tol
            ret = res[ii] <= 0
        
        if ret != 0:
            break
        
        #Save the original dual variable for daltv2d
        fun = lambda aa, info : prob.obj(aa, info, prob, ww, uu, A, B, lamdba, eta)
        
        if len(opt.eps) > 1:
            epsl = opt.eps[ii]
        
        if opt.solver in ['nt', 'ntsv']:
            aa, dfval, dgg, stat = newton(fun, aa, prob.ll, prob.uu, prob.Ac, prob.bc, epsl, prob.finddir, info, opt.display > 2)
        elif opt.solver == 'cg':
            funh = lambda xx, Hinfo = prob.hessMult(xx, A, eta, Hinfo)
            fh = {fun, funh}
            aa, dfval, dgg, stat = newton(fh, aa, prob.ll, prob.uu, prob.A, prob.bc, epsl, prob.finddir, info, opt.display > 2)
        elif opt.solver == 'qn':
            optlbfgs = {'epsginfo':epsl, 'display':opt.display - 1}
            aa, stat = lbfgs(fun, aa, prob.ll, prob.uu, prob.Ac, prob.bc, info, optlbfgs)
        elif opt.solver == 'fminunc':
            optfm = optimset('LargeScale','on','GradObj','on','Hessian'
                               , 'on','TolFun',1e-16,'TolX',0,'MaxIter',1000,'display','iter')
            aa, fvalin, exitflag = fminunc(lambda x: objdall1fminunc(xx, prob, ww, uu, A, B, lambdaa, eta, epsl)
                                               , aa, optfm)
            stat.info = info
            stat.ret = exitflag != 1
            stat.num_pcg = np.nan
        else:
            print ('Unknown solver', opt.solver)
        
        info = stat.info
        x[ii] = info.ginfo
        num_pcg[ii] = stat.num_pcg
        
        ## Update primal variable
        if prob['Aeq']:
            I1 = range(0, mm - prob.meq + 1)
            I2 = range(mm - prob.meq, mm + 1)
            gtmlp[:] = A.Ttimes(aa(I1)) + (prob.Aeq.T * aa(I2))
            ww, spec = prob.softth(ww + eta(1)*gtmp, eta(1)*lambda, info)
        else:
            ww = info.wnew
            spec = info.spec
        
        if len(uu):
            if prob.Aeq:
                uu = uu + eta(2) * (B.T * aa(range(end - prob.meq +1 )))
            else:
                uu = uu + eta(2)*(B.T * aa)
        
        # Boosting the bias term
        if len(eta) > 1:
            viol = np.hstack(( np.linalg.norm(ww - ww_old)/eta(1), np.linalg.norm(uu - uu_old)/eta(2) ))
            if opt.boostb and ii > 1 and viol(2) > viol_old * 0.5 and voil(2) > min(0.001, opt.tol):
                eta(2 = eta(2) * 20 ** (stat.ret == 0)
            viol_old = viol(2)
        
        eta = np.mul(eta, opt.eta_multp ** (stat.ret == 0))
        epsl = epsl * opt.eps_multp ** (stat.ret == 0)
        if opt.iter:
            x[:, ii + 1] = np.hstack((ww[:], uu[:]))
    
    res[ii:] = []
    fval[ii:] = []
    time[ii:] = []
    etaout[:,ii:] = []
    xi[ii:] = []
    num_pcg[ii:] = []
    
    if opt.iter:
        xx[:, ii+1:] = []
    else:
        xx = ww
    
                    
    status = {'aa':aa, 'niter':length(res), 'eta':etaout,'xi':xi,'time':time,'res':res,'opt':opt,
              'info':info, 'fval':fval, 'num_pcg':num_pcg
             }
    return xx, uu, status
        
        
    

def evalloss(prob, ww, uu, A, B):
    fnc = prob.floss
    if len(uu):
        zz = A.times(WW) + B * uu
    else:
        zz = A.times(ww)
    
    fval, gg = fnc.p(zz)
                    
    return fval, gg

## Evaluate the primal objective
def evalprim(prob, ww, uu, A, B, lambdaa):
    spec = prob.fspec(ww)
    fval = evalloss(prob, ww, uu, A, B) + lambdaa * sum(spec)
    
    if prob.Aeq:
        fval += np.linalg.norm(prob.Aeq * ww - prob.ceq)**2/tol
    
    return fval

def evaldual(prob, aa, A, B, lambdaa):
    mm = len(aa)
    fnc = prob.floss
    
    if len(B):
        aa1 = aa[I1]
        aa[I1] = aa1 - B * (B.T * B)/ (B.T * aa1)
    else:
        aa = aa - B * (B.T * B)/ (B.T * aa1)
                    
    if prob.Aeq:
        I1 = range(mm - prob.meq + 1)
        I2 = range(mm-prob.meq, mm + 1)
        vv = A.TTimes(aa[I1]) + prob.Aeq.T * aa[I2]
    else:
        vv = A.Times[aa]
                    
    dnm, ishard = prob.dnorm(vv)
                    
    if ishard and dnm > 0:
        aa = min(1, lambda.dnm) * aa
        dnm = 0
    
    dval = fnc.d(aa) + dnm
                
    return dval